In [3]:
import numpy as np
import pandas as pd
import os
import itertools
import torch
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric_temporal.nn.recurrent import EvolveGCNH, EvolveGCNO, GConvLSTM, GConvGRU
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

In [4]:
syn_data_df = pd.read_csv("datasetsrip - synthetic-prcp-SM-flood-data-v1.csv.csv")
syn_data_df.drop(['Unnamed: 0'], axis=1, inplace=True)
syn_data_df

,Prcp1,SM1,Prcp2,SM2,Prcp3,SM3,flood1,flood2,flood3
0,1.539606,0.590441,2.511432,0.474052,4.585283,0.484656,0,0,0
1,2.274125,1.249220,7.461011,2.159259,17.690303,3.474562,1,1,1
2,1.000114,0.106579,1.842560,0.010000,3.395878,0.010000,0,0,0
3,1.360013,0.429366,2.085287,0.212607,3.610686,0.137334,0,0,0
4,1.158710,0.248820,1.282190,0.010000,1.592812,0.010000,0,0,0
...,...,...,...,...,...,...,...,...,...
360,1.357030,0.426691,2.020841,0.197245,3.431358,0.103488,0,0,0
361,1.154324,0.244887,3.097722,0.201381,6.761527,0.473908,0,0,1
362,3.315756,2.183443,2.927501,2.330118,3.422077,2.060617,1,1,0
363,1.779925,0.805979,2.532323,0.718383,4.327215,0.680561,0,0,0


# 2-Layer GCN

In [10]:
def test_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        # Stack Prcp and SM for each node
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float), 
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long)
            ))
        
    class GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, 16)  
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75 
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_2layer_GCN(syn_data_df)

Epoch 1/10: Loss = 1.3124
Epoch 2/10: Loss = 0.6467
Epoch 3/10: Loss = 0.4631
Epoch 4/10: Loss = 0.3925
Epoch 5/10: Loss = 0.3635
Epoch 6/10: Loss = 0.3267
Epoch 7/10: Loss = 0.3055
Epoch 8/10: Loss = 0.2791
Epoch 9/10: Loss = 0.2669
Epoch 10/10: Loss = 0.2381
Test Accuracy: 0.9048


# EvolveGCN

In [21]:
def test_1layer_EvolveGCNH(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, node_count, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNH(node_count, dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(node_count=3, dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(20):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNH(syn_data_df)

cpu
Epoch: 000, Loss: 0.55226
Epoch: 001, Loss: 0.52784
Epoch: 002, Loss: 0.51323
Epoch: 003, Loss: 0.50421
Epoch: 004, Loss: 0.49555
Epoch: 005, Loss: 0.46541
Epoch: 006, Loss: 0.44751
Epoch: 007, Loss: 0.43158
Epoch: 008, Loss: 0.41556
Epoch: 009, Loss: 0.39790
Epoch: 010, Loss: 0.38182
Epoch: 011, Loss: 0.29999
Epoch: 012, Loss: 0.27044
Epoch: 013, Loss: 0.25071
Epoch: 014, Loss: 0.23114
Epoch: 015, Loss: 0.21194
Epoch: 016, Loss: 0.19350
Epoch: 017, Loss: 0.17608
Epoch: 018, Loss: 0.15995
Epoch: 019, Loss: 0.14563
Test Accuracy: 0.8804


In [11]:
def test_1layer_EvolveGCNO(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = EvolveGCNO(dim_in)
            self.linear = Linear(dim_in, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(20):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_EvolveGCNO(syn_data_df)

cpu
Epoch: 000, Loss: 0.57825
Epoch: 001, Loss: 0.53708
Epoch: 002, Loss: 0.52942
Epoch: 003, Loss: 0.52800
Epoch: 004, Loss: 0.52775
Epoch: 005, Loss: 0.52773
Epoch: 006, Loss: 0.52774
Epoch: 007, Loss: 0.52775
Epoch: 008, Loss: 0.52776
Epoch: 009, Loss: 0.52776
Epoch: 010, Loss: 0.52777
Epoch: 011, Loss: 0.52777
Epoch: 012, Loss: 0.52777
Epoch: 013, Loss: 0.52778
Epoch: 014, Loss: 0.52778
Epoch: 015, Loss: 0.52778
Epoch: 016, Loss: 0.52778
Epoch: 017, Loss: 0.52778
Epoch: 018, Loss: 0.52778
Epoch: 019, Loss: 0.52778
Test Accuracy: 0.5906


# 1-Layer RNN -> 1-Layer GCN 

In [5]:
def test_1layer_RNN_2layer_GCN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float), 
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float),
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long) 
            ))
        
    class RNN_GCN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(RNN_GCN, self).__init__()
            self.rnn = nn.RNN(input_size=in_channels, hidden_size=16, num_layers=1, batch_first=True)
            
            self.conv2 = GCNConv(16, out_channels)

        def forward(self, x, edge_index):
            x, h = self.rnn(x)
            x = torch.relu(x)
            x = self.conv2(x, edge_index)
            return torch.sigmoid(x)

        
        
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = RNN_GCN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75  
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    
    
    
    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)
    
    
    

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')

    del model
    torch.cuda.empty_cache()

test_1layer_RNN_2layer_GCN(syn_data_df)

C:\Users\aatal\anaconda3\envs\snakes\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1/10: Loss = 0.6120
Epoch 2/10: Loss = 0.5056
Epoch 3/10: Loss = 0.4233
Epoch 4/10: Loss = 0.3656
Epoch 5/10: Loss = 0.3155
Epoch 6/10: Loss = 0.2745
Epoch 7/10: Loss = 0.2498
Epoch 8/10: Loss = 0.2274
Epoch 9/10: Loss = 0.2124
Epoch 10/10: Loss = 0.1990
Test Accuracy: 0.9267


# 1-Layer GCN -> 1-Layer RNN 

In [20]:
def test_2layer_GCN_1layer_RNN(df, window = 3):
    df_list = [df[i:i+window] for i in range(0,df.shape[0]-window+1,1)]

    df_dict = {}
    df_dict['x'] = [df_list[i].iloc[:,0:6] for i in range(len(df_list))]
    df_dict['y'] = [df_list[i].iloc[-1,-3:] for i in range(len(df_list))]

    
    def create_node_features(df_x):
        Prcp1 = df_x['Prcp1'].values
        Prcp2 = df_x['Prcp2'].values
        Prcp3 = df_x['Prcp3'].values
        SM1 = df_x['SM1'].values
        SM2 = df_x['SM2'].values
        SM3 = df_x['SM3'].values
        node_1 = np.hstack((Prcp1, SM1))
        node_2 = np.hstack((Prcp2, SM2))
        node_3 = np.hstack((Prcp3, SM3))
        node_features = np.vstack((node_1, node_2, node_3))
        return node_features

    data_list = []
    for i in range(len(df_dict['x'])):
        data_list.append(Data(
            x=torch.tensor(create_node_features(df_dict['x'][i]), dtype=torch.float),
            y=torch.tensor(df_dict['y'][i].values, dtype=torch.float), 
            edge_index=torch.tensor([[0,2], [1,1]], dtype=torch.long) 
            ))
        
    class GCN_RNN(nn.Module):
        def __init__(self, in_channels, out_channels):
            super(GCN_RNN, self).__init__()
            self.conv1 = GCNConv(in_channels, 16)  
            self.rnn = nn.RNN(input_size=16, hidden_size=16, num_layers=1, batch_first=True)
            self.linear = nn.Linear(16, out_channels)
        
        def forward(self, x, edge_index):
            x = self.conv1(x, edge_index)
            x = torch.relu(x)
            x, h = self.rnn(x)
            x = torch.relu(x)
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN_RNN(in_channels=2*window, out_channels=1).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    train_ratio = 0.75
    train_size = int(train_ratio * len(data_list))
    train_data = data_list[:train_size]
    test_data = data_list[train_size:]

    train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=32)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            out = model(data.x, data.edge_index).squeeze()
            loss = criterion(out, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * data.num_graphs
        
        print(f'Epoch {epoch+1}/{num_epochs}: Loss = {total_loss / len(train_loader.dataset):.4f}')
    
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            data = data.to(device)
            out = model(data.x, data.edge_index).squeeze()
            predicted_labels = (out > 0.5).long()
            correct += (predicted_labels == data.y).sum().item()
            total += data.y.size(0)
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')
    
    del model
    torch.cuda.empty_cache()

test_2layer_GCN_1layer_RNN(syn_data_df)

Epoch 1/10: Loss = 0.6331
Epoch 2/10: Loss = 0.4836
Epoch 3/10: Loss = 0.3810
Epoch 4/10: Loss = 0.3237
Epoch 5/10: Loss = 0.2884
Epoch 6/10: Loss = 0.2488
Epoch 7/10: Loss = 0.2261
Epoch 8/10: Loss = 0.1993
Epoch 9/10: Loss = 0.1848
Epoch 10/10: Loss = 0.1726
Test Accuracy: 0.9121


# GConvGRU & GConvLSTM

In [18]:
def test_1layer_GConvGRU(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0,2], [1,1]], dtype=np.compat.long),
        edge_weight = np.array([1.0,1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.75)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvGRU(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x = self.recurrent(x, edge_index, edge_weight).relu()
            x = self.linear(x)
            return torch.sigmoid(x)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(10):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvGRU(syn_data_df)

cpu
Epoch: 000, Loss: 0.44646
Epoch: 001, Loss: 0.25824
Epoch: 002, Loss: 0.22243
Epoch: 003, Loss: 0.33058
Epoch: 004, Loss: 0.21400
Epoch: 005, Loss: 0.18101
Epoch: 006, Loss: 0.15226
Epoch: 007, Loss: 0.16020
Epoch: 008, Loss: 0.15111
Epoch: 009, Loss: 0.19102
Test Accuracy: 0.9239


In [19]:
def test_1layer_GConvLSTM(df):
    x = np.zeros([df.shape[0], 3, 2])
    x[:, 0, 0] = df['Prcp1'].values
    x[:, 0, 1] = df['SM1'].values
    x[:, 1, 0] = df['Prcp2'].values
    x[:, 1, 1] = df['SM2'].values
    x[:, 2, 0] = df['Prcp3'].values
    x[:, 2, 1] = df['SM3'].values

    y = np.zeros([df.shape[0], 3, 1])
    y[:, 0, 0] = df['flood1'].values
    y[:, 1, 0] = df['flood2'].values
    y[:, 2, 0] = df['flood3'].values

    dataset = StaticGraphTemporalSignal(
        edge_index = np.array([[0, 2], [1, 1]], dtype=np.compat.long),
        edge_weight = np.array([1.0, 1.0], dtype=float),
        features = x,
        targets = y
    )

    train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

    class TemporalGNN(torch.nn.Module):
        def __init__(self, dim_in):
            super().__init__()
            self.recurrent = GConvLSTM(dim_in, 16, 1)
            self.linear = Linear(16, 1)

        def forward(self, x, edge_index, edge_weight):
            x, _ = self.recurrent(x, edge_index, edge_weight)
            x = x.relu()
            x = self.linear(x)
            return x.sigmoid()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = TemporalGNN(dim_in=2).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.BCELoss().to(device)

    model.train()
    for epoch in range(10):
        cost = 0
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
            cost = criterion(y_hat, snapshot.y)
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, cost.item()))

    model.eval()
    y_pred = []
    y_true = []
    for time, snapshot in enumerate(test_dataset):
        snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_weight)
        y_pred.append(y_hat.detach().cpu().numpy())
        y_true.append(snapshot.y.detach().cpu().numpy())
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    print('Test Accuracy: {:.4f}'.format((y_pred.round() == y_true).mean()))

    del model
    torch.cuda.empty_cache()

test_1layer_GConvLSTM(syn_data_df)

cpu
Epoch: 000, Loss: 0.64018
Epoch: 001, Loss: 0.32906
Epoch: 002, Loss: 0.20860
Epoch: 003, Loss: 0.15664
Epoch: 004, Loss: 0.13061
Epoch: 005, Loss: 0.11626
Epoch: 006, Loss: 0.10767
Epoch: 007, Loss: 0.10172
Epoch: 008, Loss: 0.09704
Epoch: 009, Loss: 0.09309
Test Accuracy: 0.9178
